In [1]:
import pandas as pd
from tqdm import tqdm
from database.market import Market
from database.sim import Sim
from transformer.model_transformer import ModelTransformer
from transformer.simulation_transformer import SimulationTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from database.sec import SEC
from modeler.modeler import Modeler
import matplotlib.pyplot as plt
import warnings
import pickle
from statistics import variance
warnings.simplefilter(action='ignore', category=Warning)
from datetime import datetime, timezone, timedelta

In [2]:
sec = SEC()
market = Market("market")
sim = Sim("sim")
end = "2020-11-01"
start = "2009-01-01"
market.connect()
tickers = market.retrieve_data("sp500")
market.close()

In [3]:
sec.connect()
market.connect()
sim.connect()
sim.drop_table("sim")
sim.drop_table("errors")
reporting_gap = 365
training_years = 7
for year in tqdm(range(2010+training_years,2021)):
    for quarter in range(1,5):
        for i in (range(len(tickers["Symbol"]))):
            try:
                ticker = tickers.iloc[i]["Symbol"]
                cik = int(tickers.iloc[i]["CIK"])
                price = market.retrieve_price_data(ticker)
                funds = sec.retrieve_filing_data(cik)
                st = SimulationTransformer(ticker,start,end,reporting_gap)
                mr = ModelPreprocessor(ticker)
                mt = ModelTransformer(ticker,start,end,reporting_gap,training_years)
                data = mt.merge(price.copy(),funds.copy(),year,quarter," ")
                refined = mr.fundamental_preprocess(data)
                sp = Modeler(ticker)
                results = sp.model(refined)
                results["gap"] = reporting_gap
                result_df = pd.DataFrame([results])
                product = st.merge(price.copy(),funds.copy(),year,quarter," ")
                p_features = mr.fundamental_preprocess(product)["X"]
                product["predicted"] = pickle.loads(results["model"]).predict(p_features)
                product = product[["date","adjclose","quarter","year","predicted"]]
                product["ticker"] = ticker
                product["score"] = results["score"]
                sim.store_data("sim",product)
            except Exception as e:
                message = {"status":"modeling","year":year,"quarter":quarter,"ticker":ticker,"message":str(e)}
                sim.store_data("errors",pd.DataFrame([message]))
market.close()
sec.close()
sim.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [1:17:06<00:00, 1156.59s/it]
